### Importa Bibliotecas

In [1]:
import pandas as pd
import pickle
from sklearn.preprocessing import StandardScaler

### Parâmetros

In [2]:
data_corte = '20220901' # período validação
choques = [-10, -5, 5, 10]

### Funções

In [3]:
def aplicar_choque_regressao_linear(df, modelo, percentual_choque):
    """
    Aplica um choque de percentual_choque nas variáveis do DataFrame e realiza projeções usando um modelo de regressão linear.
    """
    df_chocado = df.copy()
    df_chocado *= (1 + percentual_choque / 100)
    
    # Adiciona a coluna de intercepto, se necessário
    if 'const' in modelo.model.exog_names:
        df_chocado = df_chocado.copy()
        df_chocado['const'] = 1

    predicoes = modelo.predict(df_chocado)
    return predicoes

In [4]:
def aplicar_choque_sarimax(df, modelo, percentual_choque):
    """
    Aplica um choque de percentual_choque nas variáveis exógenas do DataFrame e realiza projeções usando um modelo SARIMAX.
    """
    df_chocado = df.copy()
    df_chocado *= (1 + percentual_choque / 100)
    predicoes = modelo.forecast(steps=len(df_chocado), exog=df_chocado)
    return predicoes

In [5]:
def aplicar_choque_var(df, modelo, percentual_choque):
    """
    Aplica um choque de percentual_choque nas variáveis do DataFrame e realiza projeções usando um modelo VAR.
    """
    df_chocado = df.copy()
    df_chocado *= (1 + percentual_choque / 100)

    # Ajusta o modelo com os dados originais
    modelo_ajustado = modelo.fit()
    
    # Realiza a projeção com o modelo ajustado
    predicoes = modelo_ajustado.forecast(y=modelo_ajustado.endog, steps=len(df_chocado))

    # Converte as previsões em um DataFrame com as mesmas colunas do modelo
    colunas = modelo_ajustado.names  # Nomes das variáveis no modelo
    predicoes_df = pd.DataFrame(predicoes, columns=colunas)

    return predicoes_df

In [6]:
def aplicar_choque_xgboost(df, modelo, percentual_choque):
    """
    Aplica um choque de percentual_choque nas variáveis do DataFrame e realiza projeções usando um modelo XGBoost.
    """
    df_chocado = df.copy()
    
    # Normaliza as variáveis antes de aplicar o choque
    scaler = StandardScaler()
    df_normalizado = pd.DataFrame(scaler.fit_transform(df_chocado), columns=df_chocado.columns)

    # Aplica o choque
    df_normalizado *= (1 + percentual_choque / 100)

    # Inverte a normalização para retornar ao espaço original
    df_chocado = pd.DataFrame(scaler.inverse_transform(df_normalizado), columns=df_normalizado.columns)

    # Realiza a previsão
    predicoes = modelo.predict(df_chocado)
    return predicoes

### Importa Modelos

In [7]:
# modelo regressão linear
with open('modelo_linear.pkl', 'rb') as arquivo:
    reg_linear = pickle.load(arquivo)
    
# modelo sarimax
with open('modelo_sarimax_completo.pkl', 'rb') as arquivo:
    sarimax = pickle.load(arquivo)

# modelo var
with open('modelo_var_completo.pkl', 'rb') as arquivo:
    var = pickle.load(arquivo)

# modelo regressão xgboost
with open('modelo_xgb_completo.pkl', 'rb') as arquivo:
    xgb = pickle.load(arquivo)

### Importa Base

In [8]:
df = pd.read_parquet('base.parquet')

In [9]:
# cria variáveis para o modelo VAR
df['DIFF_CAMBIO_LAG_1'] = df['CAMBIO_LAG_1'].diff()
df['DIFF_CAMBIO_LAG_3'] = df['CAMBIO_LAG_3'].diff()

In [10]:
df = df[df['DATA_REFERENCIA'] > data_corte]

In [11]:
df.head()

,DATA_REFERENCIA,PD,IPCA_LAG_1,IPCA_LAG_3,IPCA_LAG_6,IPCA_LAG_12,DESEMPREGO_LAG_1,DESEMPREGO_LAG_3,DESEMPREGO_LAG_6,DESEMPREGO_LAG_12,...,SELIC_LAG_1,SELIC_LAG_3,SELIC_LAG_6,SELIC_LAG_12,CAMBIO_LAG_1,CAMBIO_LAG_3,CAMBIO_LAG_6,CAMBIO_LAG_12,DIFF_CAMBIO_LAG_1,DIFF_CAMBIO_LAG_3
93,2022-10-01,3.159615,-3.425441,-7.861667,13.488658,16.074844,8.7,9.1,10.5,12.1,...,13.623245,13.084800,10.427491,6.041083,5.2370,5.3681,4.7580,5.5400,0.0937,0.3189
94,2022-11-01,3.187230,7.315329,-4.234939,5.788194,12.015841,8.3,8.9,9.8,11.6,...,12.950555,14.979655,13.084800,7.314325,5.2503,5.1433,4.9551,5.5569,0.0133,-0.2248
95,2022-12-01,3.158198,5.031984,-3.425441,8.343738,9.119872,8.1,8.7,9.3,11.1,...,12.950555,13.623245,12.950555,9.641534,5.2747,5.2370,5.0492,5.6514,0.0244,0.0937
96,2023-01-01,3.091412,7.698614,7.315329,-7.861667,6.676334,7.9,8.3,9.1,11.2,...,14.299605,12.950555,13.084800,9.120415,5.2424,5.2503,5.3681,5.5341,-0.0323,0.0133
97,2023-02-01,3.203317,6.548183,5.031984,-4.234939,12.817282,8.4,8.1,8.9,11.2,...,14.299605,12.950555,14.979655,9.511041,5.2007,5.2747,5.1433,5.1966,-0.0417,0.0244


### Separa Target

In [12]:
df_target = df[['DATA_REFERENCIA','PD']]

### Cria bases de acordo com cada modelo

In [13]:
df_reg_linear = df[['DATA_REFERENCIA','IPCA_LAG_1','IPCA_LAG_3']]

In [14]:
df_reg_linear

,DATA_REFERENCIA,IPCA_LAG_1,IPCA_LAG_3
93,2022-10-01,-3.425441,-7.861667
94,2022-11-01,7.315329,-4.234939
95,2022-12-01,5.031984,-3.425441
96,2023-01-01,7.698614,7.315329
97,2023-02-01,6.548183,5.031984
98,2023-03-01,10.558872,7.698614
99,2023-04-01,8.861143,6.548183
100,2023-05-01,7.570845,10.558872
101,2023-06-01,2.794262,8.861143
102,2023-07-01,-0.955390,7.570845


In [15]:
df_sarimax = df[['DATA_REFERENCIA','IPCA_LAG_6']]

In [16]:
df_sarimax

,DATA_REFERENCIA,IPCA_LAG_6
93,2022-10-01,13.488658
94,2022-11-01,5.788194
95,2022-12-01,8.343738
96,2023-01-01,-7.861667
97,2023-02-01,-4.234939
98,2023-03-01,-3.425441
99,2023-04-01,7.315329
100,2023-05-01,5.031984
101,2023-06-01,7.698614
102,2023-07-01,6.548183


In [17]:
df_var = df[['DATA_REFERENCIA','PD','DESEMPREGO_LAG_12','DIFF_CAMBIO_LAG_1','DIFF_CAMBIO_LAG_3']]

In [18]:
df_var

,DATA_REFERENCIA,PD,DESEMPREGO_LAG_12,DIFF_CAMBIO_LAG_1,DIFF_CAMBIO_LAG_3
93,2022-10-01,3.159615,12.1,0.0937,0.3189
94,2022-11-01,3.187230,11.6,0.0133,-0.2248
95,2022-12-01,3.158198,11.1,0.0244,0.0937
96,2023-01-01,3.091412,11.2,-0.0323,0.0133
97,2023-02-01,3.203317,11.2,-0.0417,0.0244
98,2023-03-01,3.192858,11.1,-0.0290,-0.0323
99,2023-04-01,3.218239,10.5,0.0398,-0.0417
100,2023-05-01,3.258903,9.8,-0.1918,-0.0290
101,2023-06-01,3.286762,9.3,-0.0369,0.0398
102,2023-07-01,3.364353,9.1,-0.1312,-0.1918


In [19]:
df_xgb = df.drop(columns=['PD','DIFF_CAMBIO_LAG_1','DIFF_CAMBIO_LAG_3'])

In [20]:
df_xgb

,DATA_REFERENCIA,IPCA_LAG_1,IPCA_LAG_3,IPCA_LAG_6,IPCA_LAG_12,DESEMPREGO_LAG_1,DESEMPREGO_LAG_3,DESEMPREGO_LAG_6,DESEMPREGO_LAG_12,PIB_LAG_1,...,PIB_LAG_6,PIB_LAG_12,SELIC_LAG_1,SELIC_LAG_3,SELIC_LAG_6,SELIC_LAG_12,CAMBIO_LAG_1,CAMBIO_LAG_3,CAMBIO_LAG_6,CAMBIO_LAG_12
93,2022-10-01,-3.425441,-7.861667,13.488658,16.074844,8.7,9.1,10.5,12.1,836785.8,...,819727.6,755323.0,13.623245,13.084800,10.427491,6.041083,5.2370,5.3681,4.7580,5.5400
94,2022-11-01,7.315329,-4.234939,5.788194,12.015841,8.3,8.9,9.8,11.6,877947.5,...,826052.1,771558.9,12.950555,14.979655,13.084800,7.314325,5.2503,5.1433,4.9551,5.5569
95,2022-12-01,5.031984,-3.425441,8.343738,9.119872,8.1,8.7,9.3,11.1,879514.9,...,826025.7,782833.5,12.950555,13.623245,12.950555,9.641534,5.2747,5.2370,5.0492,5.6514
96,2023-01-01,7.698614,7.315329,-7.861667,6.676334,7.9,8.3,9.1,11.2,883886.2,...,852909.9,722909.9,14.299605,12.950555,13.084800,9.120415,5.2424,5.2503,5.3681,5.5341
97,2023-02-01,6.548183,5.031984,-4.234939,12.817282,8.4,8.1,8.9,11.2,812834.6,...,854004.1,754147.4,14.299605,12.950555,14.979655,9.511041,5.2007,5.2747,5.1433,5.1966
98,2023-03-01,10.558872,7.698614,-3.425441,21.269854,8.6,7.9,8.7,11.1,831224.5,...,836785.8,838868.0,11.616115,14.299605,13.623245,11.748906,5.1717,5.2424,5.2370,4.9684
99,2023-04-01,8.861143,6.548183,7.315329,13.488658,8.8,8.4,8.3,10.5,938679.8,...,877947.5,819727.6,14.979655,14.299605,12.950555,10.427491,5.2115,5.2007,5.2503,4.7580
100,2023-05-01,7.570845,10.558872,5.031984,5.788194,8.5,8.6,8.1,9.8,904664.8,...,879514.9,826052.1,11.616115,11.616115,12.950555,13.084800,5.0197,5.1717,5.2747,4.9551
101,2023-06-01,2.794262,8.861143,7.698614,8.343738,8.3,8.8,7.9,9.3,903238.1,...,883886.2,826025.7,14.299605,14.979655,14.299605,12.950555,4.9828,5.2115,5.2424,5.0492
102,2023-07-01,-0.955390,7.570845,6.548183,-7.861667,8.0,8.5,8.4,9.1,893074.9,...,812834.6,852909.9,13.623245,11.616115,14.299605,13.084800,4.8516,5.0197,5.2007,5.3681


### Impactos

***Regressão Linear***

In [21]:
df_linear_impacto = pd.DataFrame()

for choque in choques:
    df_linear_impacto[f"CHOQUE_{choque}%"] = aplicar_choque_regressao_linear(df_reg_linear.drop(columns='DATA_REFERENCIA'), reg_linear, choque)

df_linear_impacto = pd.concat([df_target, df_linear_impacto], axis=1)

In [22]:
df_linear_impacto

,DATA_REFERENCIA,PD,CHOQUE_-10%,CHOQUE_-5%,CHOQUE_5%,CHOQUE_10%
93,2022-10-01,3.159615,-10.994318,-11.604370,-12.824476,-13.434528
94,2022-11-01,3.187230,23.697504,25.014775,27.649316,28.966587
95,2022-12-01,3.158198,16.302932,17.209393,19.022315,19.928777
96,2023-01-01,3.091412,24.794190,26.172387,28.928783,30.306981
97,2023-02-01,3.203317,21.101833,22.274900,24.621033,25.794100
98,2023-03-01,3.192858,34.039800,35.931643,39.715328,41.607171
99,2023-04-01,3.218239,28.563411,30.151010,33.326208,34.913807
100,2023-05-01,3.258903,24.340846,25.693858,28.399882,29.752894
101,2023-06-01,3.286762,8.913916,9.409877,10.401797,10.897758
102,2023-07-01,3.364353,-3.196860,-3.373720,-3.727441,-3.904302


***SARIMAX***

In [23]:
df_sarimax_impacto = pd.DataFrame()

for choque in choques:
    df_sarimax_impacto[f"CHOQUE_{choque}%"] = aplicar_choque_sarimax(df_sarimax.drop(columns='DATA_REFERENCIA'), sarimax, choque)

In [24]:
# Garantir que os índices estejam alinhados
df_sarimax_impacto.index = df_target['DATA_REFERENCIA']

# Concatenar os DataFrames lado a lado
df_sarimax_impacto = pd.concat([df_target.set_index('DATA_REFERENCIA'), df_sarimax_impacto], axis=1)

# Resetar o índice, se necessário
df_sarimax_impacto.reset_index(inplace=True)

df_sarimax_impacto

,DATA_REFERENCIA,PD,CHOQUE_-10%,CHOQUE_-5%,CHOQUE_5%,CHOQUE_10%
0,2022-10-01,3.159615,3.278807,3.276575,3.272110,3.269878
1,2022-11-01,3.187230,3.233311,3.232353,3.230437,3.229479
2,2022-12-01,3.158198,3.205027,3.203646,3.200884,3.199503
3,2023-01-01,3.091412,3.254020,3.255321,3.257923,3.259224
4,2023-02-01,3.203317,3.317878,3.318578,3.319980,3.320681
5,2023-03-01,3.192858,3.312086,3.312653,3.313787,3.314354
6,2023-04-01,3.218239,3.221650,3.220439,3.218018,3.216807
7,2023-05-01,3.258903,3.253136,3.252303,3.250637,3.249804
8,2023-06-01,3.286762,3.257296,3.256022,3.253474,3.252200
9,2023-07-01,3.364353,3.270801,3.269717,3.267550,3.266466


***VAR***

In [25]:
df_var_impacto = pd.DataFrame()

for choque in choques:
    df_var_impacto[f"CHOQUE_{choque}%"] = aplicar_choque_var(df_var.drop(columns='DATA_REFERENCIA'), var, choque)['PD']

In [26]:
# Garantir que os índices estejam alinhados
df_var_impacto.index = df_target['DATA_REFERENCIA']

# Concatenar os DataFrames lado a lado
df_var_impacto = pd.concat([df_target.set_index('DATA_REFERENCIA'), df_var_impacto], axis=1)

# Resetar o índice, se necessário
df_var_impacto.reset_index(inplace=True)

df_var_impacto

,DATA_REFERENCIA,PD,CHOQUE_-10%,CHOQUE_-5%,CHOQUE_5%,CHOQUE_10%
0,2022-10-01,3.159615,3.143712,3.143712,3.143712,3.143712
1,2022-11-01,3.187230,3.169033,3.169033,3.169033,3.169033
2,2022-12-01,3.158198,3.199649,3.199649,3.199649,3.199649
3,2023-01-01,3.091412,3.227319,3.227319,3.227319,3.227319
4,2023-02-01,3.203317,3.251060,3.251060,3.251060,3.251060
5,2023-03-01,3.192858,3.271380,3.271380,3.271380,3.271380
6,2023-04-01,3.218239,3.288780,3.288780,3.288780,3.288780
7,2023-05-01,3.258903,3.303644,3.303644,3.303644,3.303644
8,2023-06-01,3.286762,3.316293,3.316293,3.316293,3.316293
9,2023-07-01,3.364353,3.327010,3.327010,3.327010,3.327010


***XgBoost***

In [27]:
df_xgb_impacto = pd.DataFrame()

for choque in choques:
    df_xgb_impacto[f"CHOQUE_{choque}%"] = aplicar_choque_xgboost(df_xgb.drop(columns='DATA_REFERENCIA'), xgb, choque)

In [28]:
# Garantir que os índices estejam alinhados
df_xgb_impacto.index = df_target['DATA_REFERENCIA']

# Concatenar os DataFrames lado a lado
df_xgb_impacto = pd.concat([df_target.set_index('DATA_REFERENCIA'), df_xgb_impacto], axis=1)

# Resetar o índice, se necessário
df_xgb_impacto.reset_index(inplace=True)

df_xgb_impacto

,DATA_REFERENCIA,PD,CHOQUE_-10%,CHOQUE_-5%,CHOQUE_5%,CHOQUE_10%
0,2022-10-01,3.159615,3.354156,3.354156,3.354156,3.354156
1,2022-11-01,3.187230,3.344616,3.344616,3.344616,3.344616
2,2022-12-01,3.158198,3.344616,3.344616,3.344616,3.344616
3,2023-01-01,3.091412,3.370383,3.370383,3.370383,3.370383
4,2023-02-01,3.203317,3.370383,3.370383,3.370383,3.370383
5,2023-03-01,3.192858,3.370383,3.370383,3.370383,3.370383
6,2023-04-01,3.218239,3.356012,3.356012,3.356012,3.356012
7,2023-05-01,3.258903,3.356012,3.356012,3.356012,3.356012
8,2023-06-01,3.286762,3.356012,3.356012,3.356012,3.356012
9,2023-07-01,3.364353,3.365552,3.365552,3.365552,3.365552


### Variação Média

***Linear***

In [29]:
# Criar um DataFrame para armazenar as variações médias
variacoes_medias = {}

# Iterar pelas colunas de choque
for coluna in df_linear_impacto.columns:
    if 'CHOQUE' in coluna:  # Apenas colunas de choque
        # Calcular a variação percentual média
        variacao_media = ((df_linear_impacto[coluna] - df_linear_impacto['PD']) / df_linear_impacto['PD']).mean() * 100
        variacoes_medias[coluna] = variacao_media

# Transformar o dicionário em um DataFrame para exibição
df_variacoes = pd.DataFrame(list(variacoes_medias.items()), columns=['CHOQUE', 'Variacao_Media_%'])

# Exibir o DataFrame
df_variacoes

,CHOQUE,Variacao_Media_%
0,CHOQUE_-10%,340.365607
1,CHOQUE_-5%,364.853391
2,CHOQUE_5%,413.828960
3,CHOQUE_10%,438.316745


***SARIMAX***

In [30]:
# Criar um DataFrame para armazenar as variações médias
variacoes_medias = {}

# Iterar pelas colunas de choque
for coluna in df_sarimax_impacto.columns:
    if 'CHOQUE' in coluna:  # Apenas colunas de choque
        # Calcular a variação percentual média
        variacao_media = ((df_sarimax_impacto[coluna] - df_sarimax_impacto['PD']) / df_sarimax_impacto['PD']).mean() * 100
        variacoes_medias[coluna] = variacao_media

# Transformar o dicionário em um DataFrame para exibição
df_variacoes = pd.DataFrame(list(variacoes_medias.items()), columns=['CHOQUE', 'Variacao_Media_%'])

# Exibir o DataFrame
df_variacoes

,CHOQUE,Variacao_Media_%
0,CHOQUE_-10%,0.884070
1,CHOQUE_-5%,0.861093
2,CHOQUE_5%,0.815139
3,CHOQUE_10%,0.792162


***VAR***

In [31]:
# Criar um DataFrame para armazenar as variações médias
variacoes_medias = {}

# Iterar pelas colunas de choque
for coluna in df_var_impacto.columns:
    if 'CHOQUE' in coluna:  # Apenas colunas de choque
        # Calcular a variação percentual média
        variacao_media = ((df_var_impacto[coluna] - df_var_impacto['PD']) / df_var_impacto['PD']).mean() * 100
        variacoes_medias[coluna] = variacao_media

# Transformar o dicionário em um DataFrame para exibição
df_variacoes = pd.DataFrame(list(variacoes_medias.items()), columns=['CHOQUE', 'Variacao_Media_%'])

# Exibir o DataFrame
df_variacoes

,CHOQUE,Variacao_Media_%
0,CHOQUE_-10%,1.741828
1,CHOQUE_-5%,1.741828
2,CHOQUE_5%,1.741828
3,CHOQUE_10%,1.741828


***XgBoost***

In [32]:
# Criar um DataFrame para armazenar as variações médias
variacoes_medias = {}

# Iterar pelas colunas de choque
for coluna in df_xgb_impacto.columns:
    if 'CHOQUE' in coluna:  # Apenas colunas de choque
        # Calcular a variação percentual média
        variacao_media = ((df_xgb_impacto[coluna] - df_xgb_impacto['PD']) / df_xgb_impacto['PD']).mean() * 100
        variacoes_medias[coluna] = variacao_media

# Transformar o dicionário em um DataFrame para exibição
df_variacoes = pd.DataFrame(list(variacoes_medias.items()), columns=['CHOQUE', 'Variacao_Media_%'])

# Exibir o DataFrame
df_variacoes

,CHOQUE,Variacao_Media_%
0,CHOQUE_-10%,4.115629
1,CHOQUE_-5%,4.125589
2,CHOQUE_5%,4.125589
3,CHOQUE_10%,4.125589
